# 変分量子回路を使って、高エネルギー実験でのデータ再構成のシミュレーションを行う

この実習では、**変分量子固有値ソルバー法**（*Variational Quantum Eigensolver*、VQE）と呼ばれる手法を理解し、その基本的な利用の方法を学びます。
また、VQEを用いた高エネルギー実験でのデータ再構成の実装例を紹介します。

## 内容
1. [はじめに](#introduction)
2. [量子力学における変分法](#varmethod)
    1. [数学的背景](#backgroundmath)
    2. [基底状態の下限](#groundstate)
3. [変分量子固有値ソルバー法](#vqe)
    1. [変分フォーム](#varforms)
    2. [単純な変分フォーム](#simplevarform)
    3. [パラメータ最適化](#optimization)
    4. [単一量子ビットの変分フォームの例](#example)
4. [高エネルギー実験への応用](#implementation)
    1. [量子演算の定義](#operators)
    2. [QUBO問題の導入](#qubo)
    3. [VQEによる近似解の探索](#vqe_tracking)
    4. [おまけ](#omake)
5. [参考文献](#references)

## はじめに<a id='introduction'></a>
行列で与えられる系の固有値の最小値を見つけるという操作は、多くのアプリケーションで必要となる重要な技術です。例えば化学では、分子を特徴付けるエルミート行列の最小固有値は、そのシステムの基底状態のエネルギーになります。最小の固有値を見つけるには量子位相推定アルゴリズムを使うことができますが、実用的な応用問題の実装に必要な回路はNISQ量子コンピュータでは実現できないほど長くなることが知られています。そのために、浅い量子回路を利用して分子の基底状態エネルギーを推定する手法として、**VQE**が提案されました[1]。

まずは、この関係を形式的に表現してみましょう。不明な最小固有値$\lambda_{min}$とその固有状態$|\psi_{min}\rangle$をもったエルミート行列$H$が与えられた場合、VQEは下限である$\lambda_{min}$の近似解$\lambda_{\theta}$を求めます:

\begin{align*}
    \lambda_{min} \le \lambda_{\theta} \equiv \langle \psi(\theta) |H|\psi(\theta) \rangle
\end{align*}  

ここで$|\psi(\theta)\rangle$は近似解$\lambda_{\theta}$に対応する固有状態で、$\theta$はパラメータです。つまり、このアルゴリズムでは適当な初期状態$|\psi\rangle$に$U(\theta)$で表現されるパラメータ化された回路を適用することで、$|\psi_{min}\rangle$を近似する状態$U(\theta)|\psi\rangle \equiv |\psi(\theta)\rangle$が得られます。最適なパラメータ$\theta$の値は、期待値 $\langle \psi(\theta) |H|\psi(\theta) \rangle$が最小になるように古典計算を繰り返しながら求めていくことになります。


## 量子力学における変分法<a id='varmethod'></a>
### 数学的背景<a id='backgroundmath'></a>

VQEは量子力学の変分法を用いたアプリケーションです。変分法をよりよく理解するために、基礎的な数学的背景を説明します。行列$A$の固有ベクトル$|\psi_i\rangle$とその固有値$\lambda_i$は

\begin{align*}
    A |\psi_i\rangle = \lambda_i |\psi_i\rangle
\end{align*}

の関係にあります。行列$H$がエルミート行列$H = H^{\dagger}$の場合、スペクトル定理から$H$の固有値は実数になります（$\lambda_i = \lambda_i^*$）。測定できる量は実数である必要があるため、量子システムのハミルトニアンを記述するためにはエルミート行列が適切です。さらに、$H$は以下のように表現できます。

\begin{align*}
    H = \sum_{i = 1}^{N} \lambda_i |\psi_i\rangle \langle \psi_i |
\end{align*}

ここで、各$\lambda_i$は対応する固有ベクトル$|\psi_i\rangle$の固有値です。任意の量子状態に対して観測量$H$を測定した場合の期待値は、以下の式で求められます。

\begin{align}
    \langle H \rangle_{\psi} &\equiv \langle \psi | H | \psi \rangle
\end{align}

上式の$H$を期待値の式に代入すると

\begin{align}
    \langle H \rangle_{\psi} = \langle \psi | H | \psi \rangle &= \langle \psi | \left(\sum_{i = 1}^{N} \lambda_i |\psi_i\rangle \langle \psi_i |\right) |\psi\rangle\\
    &= \sum_{i = 1}^{N} \lambda_i \langle \psi | \psi_i\rangle \langle \psi_i | \psi\rangle \\
    &= \sum_{i = 1}^{N} \lambda_i | \langle \psi_i | \psi\rangle |^2
\end{align}

になります。最後の式は、任意の状態$|\psi\rangle$に対する$H$の期待値は、$\lambda_i$を重みとした固有ベクトル$|\psi_i\rangle$と$|\psi\rangle$の内積の線形結合として与えられることを示しています。この式から、$| \langle \psi_i | \psi\rangle |^2 \ge 0$ であるために

\begin{align}
    \lambda_{min} \le \langle H \rangle_{\psi} = \langle \psi | H | \psi \rangle = \sum_{i = 1}^{N} \lambda_i | \langle \psi_i | \psi\rangle |^2
\end{align}

が成り立つことは明らかです。上記の式が**変分法**と呼ばれるもの（テキストによっては**変分原理**と呼ぶ）[2]で、$H$の最小固有値を下限として、任意の波動関数の期待値を近似的に求めることができることを表しています。この式から、$|\psi_{min}\rangle$状態の期待値は$\langle \psi_{min}|H|\psi_{min}\rangle = \langle \psi_{min}|\lambda_{min}|\psi_{min}\rangle = \lambda_{min}$になることも分かるでしょう。


### 基底状態の下限<a id='groundstate'></a>
系のハミルトニアンがエルミート行列$H$で表現されている場合、系の基底状態のエネルギーは$H$の最小固有値になります。まず$|\psi_{min}\rangle$の初期推定として任意の波動関数$|\psi \rangle$（*Ansatz*と呼ばれる）を選び、その状態での期待値$\langle H \rangle_{\psi}$を計算します。この計算を繰り返して波動関数を更新することで、ハミルトニアンの基底状態エネルギーに近い下限を得ることができます。

## 変分量子固有値ソルバー法<a id='vqe'></a>
### 変分フォーム<a id='varforms'></a>
量子コンピューター上で変分法を実装するには、系統的にAnsatzを変更する方法が必要です。VQEはこれを決まった構造を持つパラメータ化された回路の利用を使って行います。この回路はしばしば*変分フォーム（variational form）*と呼ばれ、その実行は線形変換$U(\theta)$で表現されます。変分フォームを初期状態$|\psi\rangle$（例えば、標準状態$|0\rangle$あるいはHartree Fock状態）に適用し、出力として$U(\theta)|\psi\rangle\equiv |\psi(\theta)\rangle$が生成されます。期待値$\langle \psi(\theta)|H|\psi(\theta)\rangle \approx \lambda_{min}$が出力されるように、$|\psi(\theta)\rangle$に対して繰り返しパラメータ$\theta$の最適化を行う場を提供することが変分フォームの役割です。パラメータが精度良く決められた場合、$|\psi(\theta)\rangle$は$|\psi_{min}\rangle$に十分近づくことが期待されます。

解きたい問題に応じて、ドメイン知識を取り入れるために特定の構造を持つ変分フォームを導入することがあります。ドメインに依存せず、幅広い問題への応用を可能にするような変分フォーム（例えばRyゲート）も考えられています。高エネルギー実験への応用では、このRyゲートを使った変分フォームを実装します。

### 単純な変分フォーム<a id='simplevarform'></a>
変分フォームを構築する際には、２つの相反する目的のバランスを考える必要があります。理想的には、$n$量子ビットの変分フォームは$|\psi\rangle \in \mathbb{C}^N$かつ$N=2^n$の任意の状態$|\psi\rangle$を生成できます。しかしながら、できれば可能な限り少ないパラメータで変分フォームを構築したいでしょう。

ここでは$n=1$の場合を考えます。$U3$ゲートは３つのパラメータ$\theta$、$\phi$、$\lambda$を使って以下の変換を表現します:

\begin{align}
    U3(\theta, \phi, \lambda) = \begin{pmatrix}\cos(\frac{\theta}{2}) & -e^{i\lambda}\sin(\frac{\theta}{2}) \\ e^{i\phi}\sin(\frac{\theta}{2}) & e^{i\lambda + i\phi}\cos(\frac{\theta}{2}) \end{pmatrix}
\end{align}

グローバル位相を除いて、３つのパラメータを適切に設定して実装すれば任意の単一量子ビットの変換が行えます。この*ユニバーサル*な変分フォームは３つしかパラメータがないため効率的に最適化できるという特徴があります。ただ強調すべき点は、任意の状態を生成できるということは、この変分フォームが生成する状態は$H$の期待値を計算する上で必要になる状態に限定されないということです。つまり、理想的にはこの性質によって、最小の期待値が求まるかどうかは古典計算の最適化の能力だけに依存することになります。

### パラメータ最適化<a id='optimization'></a>
パラメータ化された変分フォームを選択したら、そのパラメータをターゲットとなるハミルトニアンの期待値を最小化するように、変分法に従って最適化する必要があります。パラメータの最適化のプロセスには様々な課題があります。例えば、量子ハードウェアには様々なタイプのノイズがあり、目的関数の評価(エネルギーの計算）は実際の目的関数を反映しないかも知れません。また、いくつかのオプティマイザーはパラメーター集合の濃度(パラメーターの数)に依存して、目的関数の評価を数多く実施します。アプリケーションの要求を考慮しながら、最適なオプティマイザーを選択する必要があります。

もっとも一般的な最適化戦略は、エネルギーの変化が極大になるような方向にパラメータを更新する最急降下法です。結果として、評価の数は、最適化すべきパラメータの数に依存します。これにより、探索スペースにおいてローカル最適値をクイックに発見するアルゴリズムとなります。しかしながら、この最適化方法はしばしば局所最適時に止まることがあり、実施される回路評価数によっては比較的時間がかかります。直感的な最適化戦略ですが、VQEで利用するにはお勧めできません。

ノイズのある目的関数を最適化する適切なオプティマイザーとして、*Simultaneous Perturbation Stochastic Approximation* オプティマイザー (SPSA)があります。SPSAは２回の測定だけで、目的関数の勾配を近似します。最急降下法では各パラメータを独立に摂動させるのに対して、全てのパラメータを同時にランダムに摂動させます。VQEをノイズ込みのシミュレーター、もしくは実ハードウェアで利用する場合には、古典オプティマイザーとして SPSA が推奨されます。

コスト関数の評価にノイズがない場合（例えば、VQEを状態ベクトルシミュレーターで利用する場合など）は、多様な古典オプティマイザーを利用できます。Qiskit Aqua でサポートされている２つのオプティマイザーは、*Sequential Least Squares Programming* オプティマイザー (SLSQP) と *Constrained Optimization by Linear Approximation* オプティマイザー (COBYLA) です。COBYLAでは、目的関数の評価を最適化の繰り返しで１回のみ実施（つまり評価の数はパラメータセットの濃度には依存しない）ということに着目します。従って、目的関数がノイズがない場合、及び評価の数を最小化したい場合は、COBYLAの利用がお勧めです。

### 単一量子ビットの変分フォームの例<a id='example'></a>
ではここで、基底エネルギー推定問題と似たような単一量子ビットの変分フォームを利用してみましょう。問題は、ランダムな確率ベクトルが$\vec{x}$ が与えられており、出力の確率分布が $\vec{x}$ に近くなるように、可能な単一量子ビット変分フォームのパラメーターを決定します（ここで近くは２つの確率ベクトル間のマンハッタン距離によって定義します）。

最初に python でランダム確率ベクトルを作成します。

In [1]:
import numpy as np
np.random.seed(999999)
target_distr = np.random.rand(2)
# We now convert the random vector into a valid probability vector
target_distr /= sum(target_distr)

次に、単一の U3 変分フォームの３つのパラメーターを引数として受け取り、対応する量子回路をリターンする関数を定義します:

In [2]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
def get_var_form(params):
    qr = QuantumRegister(1, name="q")
    cr = ClassicalRegister(1, name='c')
    qc = QuantumCircuit(qr, cr)
    qc.u3(params[0], params[1], params[2], qr[0])
    qc.measure(qr, cr[0])
    return qc

また、変分フォームのパラメータのリストを入力とし、パラメータに対応したコストを計算する目的関数を定義します:

In [3]:
from qiskit import Aer, execute
backend = Aer.get_backend("qasm_simulator")
NUM_SHOTS = 10000

def get_probability_distribution(counts):
    output_distr = [v / NUM_SHOTS for v in counts.values()]
    if len(output_distr) == 1:
        output_distr.append(0)
    return output_distr

def objective_function(params):
    # Obtain a quantum circuit instance from the paramters
    qc = get_var_form(params)
    # Execute the quantum circuit to obtain the probability distribution associated with the current parameters
    result = execute(qc, backend, shots=NUM_SHOTS).result()
    # Obtain the counts for each measured state, and convert those counts into a probability vector
    output_distr = get_probability_distribution(result.get_counts(qc))
    # Calculate the cost as the distance between the output distribution and the target distribution
    cost = sum([np.abs(output_distr[i] - target_distr[i]) for i in range(2)])
    return cost

最後に、COBYLA オプティマイザーのインスタンスを作成し、アルゴリズムを実行します。出力は実行の度に異なることに注意してください。また、近いとはいえ、得られた分布はターゲットの分布とは完全に同じではありません。しかしながら、ショットの数を増やすことで出力の確度を向上させることができるでしょう。

In [4]:
from qiskit.aqua.components.optimizers import COBYLA

# Initialize the COBYLA optimizer
optimizer = COBYLA(maxiter=500, tol=0.0001)

# Create the initial parameters (noting that our single qubit variational form has 3 parameters)
params = np.random.rand(3)
ret = optimizer.optimize(num_vars=3, objective_function=objective_function, initial_point=params)

# Obtain the output distribution using the final parameters
qc = get_var_form(ret[0])
counts = execute(qc, backend, shots=NUM_SHOTS).result().get_counts(qc)
output_distr = get_probability_distribution(counts)

print("Target Distribution:", target_distr)
print("Obtained Distribution:", output_distr)
print("Output Error (Manhattan Distance):", ret[1])
print("Parameters Found:", ret[0])


Target Distribution: [0.51357006 0.48642994]
Obtained Distribution: [0.5353, 0.4647]
Output Error (Manhattan Distance): 0.020459881261160884
Parameters Found: [ 1.52294877 -0.07797282  0.65499835]


## 高エネルギー実験への応用<a id='implementation'></a>
このセクションでは、高エネルギー実験へのVQEの応用を考えます。特に、高エネルギー粒子の衝突で発生する**荷電粒子の飛跡を再構成する**こと（一般的に**Tracking**と呼ばれます）への応用を考えます。
データとしては、TrackMLチャレンジで提供されたオープンデータを活用します。

まず、必要なライブラリを最初にインポートします。


In [5]:
import numpy as np
import matplotlib.pyplot as plt

from qiskit import Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.circuit.library import TwoLocal
from qiskit.aqua.algorithms import VQE, NumPyMinimumEigensolver, NumPyEigensolver
from qiskit.optimization.applications.ising.common import sample_most_likely
from qiskit.aqua.components.optimizers import SPSA, COBYLA
from qiskit.aqua import QuantumInstance

### 量子演算の定義<a id='operators'></a>

次に、VQEでTrackingを実行するのに必要な量子演算を定義します。ここではシンプルな1次元のイジング模型のハミルトニアンを考え、その元での最小固有値を求める問題としてTrackingを実行します。

In [6]:
from qiskit.quantum_info import Pauli
from qiskit.aqua.operators import WeightedPauliOperator

def get_ising_qubitops(J_ij,h_i,penalty=1e6):
    """Generate Hamiltonian for Ising model.

    Args:
        J_ij, h_ij : Coefficients of the couplings and linear terms
        penalty (float) : Penalty coefficient for the constraints

    Returns:
        operator.Operator, float: operator for the Hamiltonian and a
        constant shift for the obj function.
    """

    num_qubits = len(h_i)
    print("Number of qubits (selected triplets) =",num_qubits)
    pauli_list = []
    shift = 0

    zero = np.zeros(num_qubits, dtype=np.bool)
    for i in range(num_qubits):
        for j in range(num_qubits):
            if i >= j:
                continue
            if J_ij[i][j] == 0:
                continue
            shift += J_ij[i][j]
            vp = np.zeros(num_qubits, dtype=np.bool)
            vp[i] = True
            vp[j] = True
            pauli_list.append([J_ij[i][j], Pauli(vp, zero)])

    zero = np.zeros(num_qubits, dtype=np.bool)
    for i in range(num_qubits):
        shift += h_i[i]
        vp = np.zeros(num_qubits, dtype=np.bool)
        vp[i] = True
        pauli_list.append([h_i[i], Pauli(vp, zero)])

    return WeightedPauliOperator(paulis=pauli_list), shift

$J_{ij}$は量子ビット$i$と$j$の間の相互作用の強さを表す係数で、$h_i$は$i$量子ビットに掛かる磁場の強さを表します。これらの係数をどのようにして決めるのかは、以下で説明します。量子演算としては、$J_{ij}$や$h_i$が係数として掛かったパウリ$Z$演算子(to check)を使って記述しています。

次に計算に用いるファイルを読み込んで、そこから$J_{ij}$と$h_i$を決定します。

In [7]:
file_r = 'data_files/QUBO_05pct_input.txt'
from ast import literal_eval
with open(file_r, "r") as f:
    line = f.read()
    Q = literal_eval(line)
print("Q size =",len(Q))


n_qubits = 100

nvar = 0
key_i = []
b_ij = np.zeros((n_qubits,n_qubits))
for (k1, k2), v in Q.items():
    if k1 == k2:
        b_ij[nvar][nvar] = v
        key_i.append(k1)
        nvar += 1

for (k1, k2), v in Q.items():
    if k1 != k2:
        for i in range(nvar):
            for j in range(nvar):
                if k1 == key_i[i] and k2 == key_i[j]:
                    if i < j:
                        b_ij[i][j] = v
                    else:
                        b_ij[j][i] = v

この実習では、**QUBO**（*Quadratic Unconstrained Binary Optimization*、2次制約無し2値最適化）と呼ばれる問題として与えられたモデルをVQEで解くことを考えます。QUBOはバイナリー値（0か1）で定義されますが、シンプルな変換でパウリ$Z$の固有値（-1か1）としても表現できるため、ここでは同等の問題を扱っていると考えてください。このQUBOの係数$b_{ij}$を求める部分は省略しますが、実際はいくつかの前処理を行って求めることになります。

### QUBO問題の導入<a id='qubo'></a>

説明を追加???

パウリ$Z$の固有値を使って問題を解くため、以下では係数の変換を行います。

In [8]:
J_ij = np.zeros((nvar,nvar))
for i in range(nvar):
    for j in range(nvar):
        if i >= j:
            continue
        J_ij[i][j] = b_ij[i][j]
        if J_ij[i][j] == 0:
            continue

h_i = np.zeros(nvar)
for i in range(nvar):
    bias = 0
    for k in range(n_qubits):
        bias += b_ij[i][k]+b_ij[k][i]
    bias *= -1
    h_i[i] = bias
    if h_i[i] == 0:
        continue

### VQEによる近似解の探索<a id='vqe_tracking'></a>

ここまでは準備段階で、ここからVQEを使ってエネルギーの最小固有値（の近似解）を求めていくことになります。ただその前に、このハミルトニアンの行列を対角化して厳密にエネルギーの最小固有値とその固有ベクトルを求めた場合の答えを出してみましょう。

In [9]:
# === Solving Ising problem =====================
qubitOp, offset = get_ising_qubitops(J_ij,h_i)
print("")
print("total number of qubits = ",qubitOp.num_qubits)

# Making the Hamiltonian in its full form and getting the lowest eigenvalue and eigenvector
ee = NumPyMinimumEigensolver(qubitOp)
result = ee.run()

print('Eigensolver: objective =', result.eigenvalue.real+offset)
x = sample_most_likely(result.eigenstate)
print('Eigensolver: x =',x)

samples_eigen = {}
for i in range(nvar):
    samples_eigen[key_i[i]] = x[i]

xのリストで1になっている量子ビットが最小エネルギーに対応するものとして選ばれているのが分かります。

次に、同じハミルトニアンモデルをVQEに実装して、最小エネルギーを求めてみます。オプティマイザーとしてはSPSAあるいはCOBYLAを使っています。オプティマイザーの繰り返し回数に依りますが、上で求めた厳密解と(ほぼ)同じ結果が得られることが分かると思います。

In [10]:
# --- run optimization with VQE
seed = 10598
spsa = SPSA(maxiter=300)
cobyla = COBYLA(maxiter=500)
two = TwoLocal(qubitOp.num_qubits, 'ry', 'cz', 'linear', reps=1)
print(two)

#backend = Aer.get_backend('statevector_simulator')
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend=backend, shots=1024, seed_simulator=seed)
vqe = VQE(qubitOp, two, spsa)
#vqe = VQE(qubitOp, two, cobyla)
result = vqe.run(quantum_instance)

print('')
#print(result['optimal_parameters'])
print('VQE: objective =', result.eigenvalue.real+offset)
x = sample_most_likely(result.eigenstate)
print('VQE x =',x)

samples_vqe = {}
for i in range(nvar):
    samples_vqe[key_i[i]] = x[i]

### おまけ<a id='omake'></a>

Trackingがうまく行っても、この答えでは面白くないですよね。正しく飛跡が見つかったかどうか目で確認するため、以下のコードを走らせてみましょう。このコードは、QUBOを定義する時に使った検出器のヒット位置をビーム軸に垂直な平面でプロットして、どのヒットが選ばれたかを分かりやすく可視化したものです。緑の線が実際に見つかった飛跡で、青の線を含めたものが全体の飛跡の候補です。この実習では限られた数の量子ビットしか使っていないため、大部分の飛跡は見つけられていませんが、緑の線から計算に使った３点ヒットからは正しく飛跡が見つかっていることが分かると思います。

In [11]:
from hepqpr.qallse import *
input_path = './data_files/event000001000-hits.csv'
dw = DataWrapper.from_path(input_path)

# get the results
#all_doublets = Qallse.process_sample(samples_eigen)
all_doublets = Qallse.process_sample(samples_vqe)

final_tracks, final_doublets = TrackRecreaterD().process_results(all_doublets)
#print("all_doublets =",all_doublets)
#print("final_tracks =",final_tracks)
#print("final_doublets =",final_doublets)

p, r, ms = dw.compute_score(final_doublets)
trackml_score = dw.compute_trackml_score(final_tracks)

print(f'SCORE  -- precision (%): {p * 100}, recall (%): {r * 100}, missing: {len(ms)}')
print(f'          tracks found: {len(final_tracks)}, trackml score (%): {trackml_score * 100}')

from hepqpr.qallse.plotting import iplot_results, iplot_results_tracks
dims = ['x', 'y']
_, missings, _ = diff_rows(final_doublets, dw.get_real_doublets())
dout = 'plot-ising_found_tracks.html'
iplot_results(dw, final_doublets, missings, dims=dims, filename=dout)


## 参考文献<a id='references'></a>
1. Peruzzo, Alberto, et al. "A variational eigenvalue solver on a photonic quantum processor." *Nature communications* 5 (2014): 4213.
2. Griffiths, David J., and Darrell F. Schroeter. Introduction to quantum mechanics. *Cambridge University Press*, 2018.
3. Shende, Vivek V., Igor L. Markov, and Stephen S. Bullock. "Minimal universal two-qubit cnot-based circuits." arXiv preprint quant-ph/0308033 (2003).
4. Kandala, Abhinav, et al. "Hardware-efficient variational quantum eigensolver for small molecules and quantum magnets." Nature 549.7671 (2017): 242.